### Install the `anvil-uplink` library, using `!pip install anvil-uplink` in the first cell of the notebook:

In [1]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.5 MB/s 
     |████████████████████████████████| 51 kB 166 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=c1d0171f22b82bb7f06bb9333e9ae5ec810cfcccb82dcd6f678c503595f42d00
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


### Importing `anvil.server` means, when this notebook is connected via the Uplink, it will behave in the same way as any other [Anvil Server Module](https://anvil.works/docs/server).

In [1]:
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2
from matplotlib import pyplot as plt
from tensorflow import keras as ks
from tqdm.auto import tqdm
import tensorflow as tf
from PIL import Image
import numpy as np
import anvil.server
import anvil.media
import h5py
import pickle
import keras
import math
import cv2

### Then connect this notebook to your app using your Uplink key `anvil.server.connect("your-uplink-key")`:

In [2]:
anvil.server.connect("server_E7YX2EF6WHHJUN2C7N7KFRJX-I7YYVPXW3JN4MCCJ")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -q "/content/drive/Shareddrives/MIRCV Project/archive.zip"
!unzip -q "/content/drive/Shareddrives/MIRCV Project/mirflickr25k.zip"

In [5]:
% cd "/content/drive/Shareddrives/MIRCV Project/index/"

/content/drive/Shareddrives/MIRCV Project/index


In [6]:
% run index_IVFADC.ipynb

In [7]:
with open("/content/drive/Shareddrives/MIRCV Project/Performance Evaluation/index_512.pkl", "rb") as fp:  # Unpickling
  index_512 = pickle.load(fp)
with open("/content/drive/Shareddrives/MIRCV Project/Performance Evaluation/index_512_FT.pkl", "rb") as fp:  # Unpickling
  index_512_FT = pickle.load(fp)
with open("/content/drive/Shareddrives/MIRCV Project/Performance Evaluation/total_dataframe.pkl", "rb") as fp:  # Unpickling
  total_dataframe = pickle.load(fp)
with open("/content/drive/Shareddrives/MIRCV Project/pickles queries/class_names.pkl", "rb") as fp:  # Unpickling
  class_names = pickle.load(fp)

In [8]:
def plot_images(indexes, limit):
  print(indexes)
  rows = math.ceil(limit/5)
  columns = 5
  fig = plt.figure(figsize=(columns*4,rows*4))
  for i, id in enumerate(indexes[:limit]):
    img = Image.open(indexes[i])
    fig.add_subplot(rows, columns, i+1, )
    label = str(indexes[i])
    if "mirflickr" in label:
      label = "Distractor"
    else:
      label = label.split("/")[4]
    title = str(i+1) + ") Species: " + label
    plt.title(title)
    plt.imshow(img)
    plt.axis('off')

  fig.tight_layout()
  #set path where save the image with results to show in Anvil
  plt.savefig('result.png')

In [9]:
model = tf.keras.models.load_model('/content/drive/Shareddrives/MIRCV Project/ResNet152v2/EXP/ResNet152v2_exp3.h5')
feature_extractor_512 = keras.Model(inputs=model.inputs, outputs=model.get_layer('dense').output)

In [10]:
model_finetuned = tf.keras.models.load_model('/content/drive/Shareddrives/MIRCV Project/ResNet152v2/EXP/FineTuning_EXP3.h5')
feature_extractor_512_FT = keras.Model(inputs=model_finetuned.inputs, outputs=model_finetuned.get_layer('dense').output)

In [11]:
import os
fpath = '/content/images/Images/'
categories = sorted(os.listdir(fpath))
breeds = []
for cat in categories:
  x = cat.split("-")[1]
  os.rename(fpath + cat, fpath + x)
  breeds.append(x)

breeds = sorted(breeds)

In [12]:
@anvil.server.callable
def perform_query(image_file, n_probe, k, metric, ft):

  if metric=='Cosine similarity':
    metric='cosine'
  else:
    metric='sqeuclidean'

  if ft:
    mod=model_finetuned
    index=index_512_FT
    fe=feature_extractor_512_FT
  else:
    mod=model
    index=index_512
    fe=feature_extractor_512
  
  with anvil.media.TempFile(image_file) as f:
    image = tf.keras.preprocessing.image.load_img(f, target_size=(224,224))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    img_features = fe.predict(input_arr, verbose=1)


  class_label = mod.predict(input_arr)
  class_label = np.argmax(class_label, axis=-1)
  
  class_label = class_names[class_label[0]].split("-")[1]

  q=np.zeros(1)
  q = img_features
 
  I, D, search_time = index.search(q, k, n_probe, metric)
  ranked_labels = [total_dataframe.loc[int(x), 'Path'] for x in I.ravel()]
  plot_images(ranked_labels, k)
  obj = anvil.media.from_file('/content/drive/Shareddrives/MIRCV Project/index/result.png', 'image/png')
  return obj, class_label


### Finally, let's add `anvil.server.wait_forever()` function so the notebook is always available to the web app:

In [ ]:
anvil.server.wait_forever()

1/1 [==============================] - 3s 3s/step
['/content/images/Images/Saint_Bernard/n02109525_9869.jpg', '/content/images/Images/Saint_Bernard/n02109525_2943.jpg', '/content/images/Images/Saint_Bernard/n02109525_7982.jpg', '/content/images/Images/Saint_Bernard/n02109525_368.jpg', '/content/images/Images/Leonberg/n02111129_1380.jpg', '/content/images/Images/Saint_Bernard/n02109525_13627.jpg', '/content/images/Images/Saint_Bernard/n02109525_15746.jpg', '/content/images/Images/Leonberg/n02111129_1661.jpg', '/content/images/Images/Saint_Bernard/n02109525_6931.jpg', '/content/images/Images/Saint_Bernard/n02109525_13780.jpg', '/content/images/Images/Saint_Bernard/n02109525_10908.jpg', '/content/images/Images/Leonberg/n02111129_426.jpg', '/content/images/Images/Saint_Bernard/n02109525_8417.jpg', '/content/images/Images/Saint_Bernard/n02109525_9130.jpg', '/content/images/Images/Saint_Bernard/n02109525_10032.jpg', '/content/images/Images/Saint_Bernard/n02109525_1480.jpg', '/content/images/

KeyboardInterrupt: ignored